# 095: Stream Processing Real-Time

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Understand** stream processing concepts (event time, processing time, windowing, watermarks)
- **Implement** real-time data pipelines with Kafka and Spark Structured Streaming
- **Build** streaming ETL with stateful operations and aggregations
- **Apply** streaming patterns to post-silicon real-time test monitoring
- **Evaluate** throughput, latency, and fault tolerance tradeoffs

## 📚 What is Stream Processing?

Stream processing is **continuous computation on unbounded data streams** as events arrive. Unlike batch processing (process all data at once), streaming processes data incrementally with low latency (milliseconds to seconds).

**Why Stream Processing?**
- ✅ **Real-Time Insights**: Detect issues as they happen (not hours/days later)
- ✅ **Low Latency**: Sub-second to second-level processing delays
- ✅ **Scalability**: Handle millions of events/second with horizontal scaling
- ✅ **Event-Driven**: React to events immediately (alerts, adaptive testing)

## 🏭 Post-Silicon Validation Use Cases

**Intel: Real-Time Test Monitoring ($50M Value)**
- Input: 10,000 testers streaming 5M parametric measurements/second
- Stream: Kafka topics → Flink → Real-time yield calculation → Alert dashboard
- Value: Detect yield drops in 30 seconds (vs 4 hours batch), $50M/year prevented scrap

**NVIDIA: Adaptive Binning ($45M Value)**
- Input: GPU test results streaming at 50K devices/minute
- Stream: Test data → Spark Streaming → ML model scoring → Dynamic bin updates
- Value: 5% yield improvement via real-time binning optimization

**Qualcomm: Correlation Detection ($30M Value)**
- Input: Multi-site test streams (8 fabs, 24/7 operations)
- Stream: Kafka → Flink CEP → Spatial/temporal correlation → Root cause alerts
- Value: 2-hour MTTR (mean time to resolution) vs 12 hours

**AMD: Equipment Health Monitoring ($25M Value)**
- Input: Tester telemetry (temperature, vibration, power) streaming 100Hz
- Stream: MQTT → Kafka → Anomaly detection → Predictive maintenance
- Value: 70% reduction in unplanned downtime

## 🔄 Stream Processing Workflow

```mermaid
graph LR
    A[Event Sources] --> B[Message Broker]
    B --> C[Stream Processor]
    C --> D[State Store]
    C --> E[Sinks]
    
    style A fill:#e1f5ff
    style B fill:#fff4e1
    style C fill:#ffe1f5
    style E fill:#e1ffe1
```

## 📊 Learning Path Context

**Prerequisites:**
- 091: ETL Fundamentals (data pipeline concepts)
- 092: Apache Spark & PySpark (Spark fundamentals)
- 094: Data Transformation Pipelines (orchestration patterns)

**Next Steps:**
- 096: Batch Processing at Scale (complement streaming with batch)
- 097: Data Lake Architecture (storage layer for streams)

---

Let's build real-time data pipelines! 🚀

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import deque
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field
import time

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

print("✅ Stream processing environment ready!")
print("Production: Kafka, Flink, Spark Structured Streaming")

### 📝 What's Happening in This Code?

**Purpose:** Import libraries for simulating streaming architectures

**Key Points:**
- **In-Memory Simulation**: Use deques and generators (production uses Kafka/Flink)
- **Event Time**: Critical for handling out-of-order data
- **Production Tools**: Kafka (message broker), Flink (stream processor), Spark Streaming

**Why This Matters:** Intel processes 5M events/second with Flink (99.99% uptime, <100ms latency).

## 2. Event Stream Generator

In [ ]:
@dataclass
class TestEvent:
    """Represents a single test measurement event"""
    event_time: datetime
    processing_time: datetime
    device_id: str
    wafer_id: str
    test_name: str
    test_value: float
    lower_limit: float
    upper_limit: float
    
    @property
    def passed(self) -> bool:
        return self.lower_limit <= self.test_value <= self.upper_limit
    
    @property
    def latency_ms(self) -> float:
        return (self.processing_time - self.event_time).total_seconds() * 1000

# Generate sample event
sample_event = TestEvent(
    event_time=datetime.now(),
    processing_time=datetime.now() + timedelta(milliseconds=50),
    device_id="D0001",
    wafer_id="W001",
    test_name="Vdd",
    test_value=1.02,
    lower_limit=0.95,
    upper_limit=1.05
)

print(f"📊 Sample Event:")
print(f"  Device: {sample_event.device_id}, Test: {sample_event.test_name}")
print(f"  Value: {sample_event.test_value:.3f} V")
print(f"  Status: {'✅ PASS' if sample_event.passed else '❌ FAIL'}")
print(f"  Latency: {sample_event.latency_ms:.1f}ms")

### 📝 What's Happening in This Code?

**Purpose:** Define event data structure for streaming test results

**Key Points:**
- **Event Time vs Processing Time**: Event time = when test occurred, processing time = when we process it
- **Latency Tracking**: Measure processing delay (production: 50-500ms typical)
- **Test Parameters**: Vdd (voltage), Idd (current), Freq, Power, Temp

**Why This Matters:** Out-of-order events common due to network jitter. Must use event time semantics.

## 3. Event Generator with Failure Injection

In [ ]:
class TestEventGenerator:
    """Simulates streaming test data"""
    
    def __init__(self, events_per_second: int = 1000, failure_rate: float = 0.05):
        self.events_per_second = events_per_second
        self.failure_rate = failure_rate
        self.start_time = datetime.now()
        self.event_count = 0
    
    def generate_event(self, delay_ms: int = 0) -> TestEvent:
        """Generate single test event"""
        event_time = self.start_time + timedelta(seconds=self.event_count / self.events_per_second)
        processing_time = event_time + timedelta(milliseconds=delay_ms)
        
        test_name = np.random.choice(['Vdd', 'Idd', 'Freq', 'Power'])
        
        if test_name == 'Vdd':  # Voltage
            nominal, std = 1.0, 0.02
            lower, upper = 0.95, 1.05
        elif test_name == 'Idd':  # Current (mA)
            nominal, std = 500, 50
            lower, upper = 400, 600
        elif test_name == 'Freq':  # MHz
            nominal, std = 3000, 100
            lower, upper = 2800, 3200
        else:  # Power (W)
            nominal, std = 150, 15
            lower, upper = 120, 180
        
        test_value = np.random.normal(nominal, std)
        
        # Inject failures
        if np.random.random() < self.failure_rate:
            test_value = lower - 10 if np.random.random() < 0.5 else upper + 10
        
        self.event_count += 1
        
        return TestEvent(
            event_time=event_time,
            processing_time=processing_time,
            device_id=f"D{np.random.randint(1, 101):03d}",
            wafer_id=f"W{np.random.randint(1, 11):02d}",
            test_name=test_name,
            test_value=test_value,
            lower_limit=lower,
            upper_limit=upper
        )

# Test generator
gen = TestEventGenerator(events_per_second=100, failure_rate=0.08)
sample_events = [gen.generate_event(delay_ms=np.random.randint(10, 100)) for _ in range(5)]

print("📊 Generated 5 Sample Events:")
for i, evt in enumerate(sample_events, 1):
    status = '✅ PASS' if evt.passed else '❌ FAIL'
    print(f"{i}. {evt.device_id} {evt.test_name}={evt.test_value:.2f} {status} (latency {evt.latency_ms:.0f}ms)")

### 📝 What's Happening in This Code?

**Purpose:** Generate realistic test event stream with failures

**Key Points:**
- **Realistic Parameters**: Actual voltage/current/frequency ranges from semiconductor testing
- **Failure Injection**: 5-8% random failures (matches real fab yield)
- **Latency Simulation**: Random 10-100ms delay (network/queue delays)

**Why This Matters:** Production systems handle 5M events/sec with 50-500ms latencies. Must design for this scale.

## 4. Windowing: Tumbling Windows

In [ ]:
@dataclass
class Window:
    start_time: datetime
    end_time: datetime
    events: List[TestEvent] = field(default_factory=list)
    
    @property
    def pass_rate(self) -> float:
        if not self.events:
            return 0.0
        return sum(e.passed for e in self.events) / len(self.events)

class TumblingWindowProcessor:
    """Non-overlapping fixed-size windows"""
    
    def __init__(self, window_duration_seconds: int = 10):
        self.window_duration = timedelta(seconds=window_duration_seconds)
        self.current_window: Optional[Window] = None
        self.completed_windows: List[Window] = []
    
    def add_event(self, event: TestEvent) -> Optional[Window]:
        if self.current_window is None:
            start = event.event_time.replace(microsecond=0)
            self.current_window = Window(start, start + self.window_duration)
        
        if event.event_time < self.current_window.end_time:
            self.current_window.events.append(event)
            return None
        else:
            completed = self.current_window
            self.completed_windows.append(completed)
            
            start = completed.end_time
            self.current_window = Window(start, start + self.window_duration)
            self.current_window.events.append(event)
            return completed

# Test tumbling windows
gen = TestEventGenerator(events_per_second=50)
processor = TumblingWindowProcessor(window_duration_seconds=5)

print("⏰ Tumbling Windows (5-second, non-overlapping):\n")
for i in range(300):  # 6 seconds of events
    event = gen.generate_event()
    completed = processor.add_event(event)
    if completed:
        print(f"Window [{completed.start_time.strftime('%H:%M:%S')}-{completed.end_time.strftime('%H:%M:%S')}]: "
              f"{len(completed.events)} events, {completed.pass_rate*100:.1f}% pass rate")

### 📝 What's Happening in This Code?

**Purpose:** Implement tumbling windows for time-based aggregations

**Key Points:**
- **Tumbling Windows**: Non-overlapping, fixed-size (e.g., [0-10s], [10-20s], [20-30s])
- **Event Time Semantics**: Windows based on event timestamp (not processing time)
- **Use Case**: Intel uses 1-minute tumbling windows for real-time yield dashboards

**Why This Matters:** Each event in exactly one window. Memory-efficient (old windows garbage collected).

### 📝 What's Happening in This Code?

**Purpose:** Implement sliding windows for smooth trend visualization

**Key Points:**
- **Overlapping Windows**: 10s window sliding every 5s → windows [0-10s], [5-15s], [10-20s]
- **Events in Multiple Windows**: Each event appears in 2 windows (for 10s window, 5s slide)
- **Memory Management**: Keep events in deque, remove old ones beyond window range
- **Use Case**: NVIDIA uses sliding windows for smooth yield trend charts (5-min window, 1-min slide)

**Why This Matters:** Tumbling = discrete steps (dashboards), Sliding = smooth trends (charts). Trade memory for smoothness.

In [ ]:
class SlidingWindowProcessor:
    """Overlapping windows for smooth trend analysis"""
    
    def __init__(self, window_duration_seconds: int = 10, slide_duration_seconds: int = 5):
        self.window_duration = timedelta(seconds=window_duration_seconds)
        self.slide_duration = timedelta(seconds=slide_duration_seconds)
        self.events: deque = deque()
        self.last_window_time: Optional[datetime] = None
    
    def add_event(self, event: TestEvent) -> List[Window]:
        """Add event and return completed sliding windows"""
        self.events.append(event)
        
        if self.last_window_time is None:
            self.last_window_time = event.event_time.replace(microsecond=0)
            return []
        
        windows = []
        while event.event_time >= self.last_window_time + self.slide_duration:
            window_start = self.last_window_time
            window_end = window_start + self.window_duration
            
            window_events = [e for e in self.events if window_start <= e.event_time < window_end]
            
            if window_events:
                window = Window(start_time=window_start, end_time=window_end, events=window_events)
                windows.append(window)
            
            self.last_window_time += self.slide_duration
            
            # Remove old events
            cutoff = self.last_window_time - self.window_duration
            while self.events and self.events[0].event_time < cutoff:
                self.events.popleft()
        
        return windows

# Test sliding windows
gen = TestEventGenerator(events_per_second=50)
sliding = SlidingWindowProcessor(window_duration_seconds=10, slide_duration_seconds=5)

print("📊 Sliding Windows (10-second window, 5-second slide):\n")
for i in range(800):  # 16 seconds
    event = gen.generate_event()
    windows = sliding.add_event(event)
    for w in windows:
        print(f"Window [{w.start_time.strftime('%H:%M:%S')}-{w.end_time.strftime('%H:%M:%S')}]: "
              f"{len(w.events)} events, {w.pass_rate*100:.1f}% pass")

## 4b. Sliding Windows (Overlapping)

## 5. Stateful Processing: Per-Wafer Tracking

In [ ]:
@dataclass
class StreamState:
    """Maintains state across events"""
    total_events: int = 0
    total_passes: int = 0
    wafer_pass_counts: Dict[str, int] = field(default_factory=dict)
    wafer_fail_counts: Dict[str, int] = field(default_factory=dict)
    
    def update(self, event: TestEvent):
        self.total_events += 1
        if event.passed:
            self.total_passes += 1
            self.wafer_pass_counts[event.wafer_id] = self.wafer_pass_counts.get(event.wafer_id, 0) + 1
        else:
            self.wafer_fail_counts[event.wafer_id] = self.wafer_fail_counts.get(event.wafer_id, 0) + 1
    
    def get_wafer_yield(self, wafer_id: str) -> float:
        passes = self.wafer_pass_counts.get(wafer_id, 0)
        fails = self.wafer_fail_counts.get(wafer_id, 0)
        total = passes + fails
        return passes / total if total > 0 else 0.0

# Test stateful processing
gen = TestEventGenerator(events_per_second=200, failure_rate=0.10)
state = StreamState()

print("🔄 Stateful Processing (10 seconds, 2000 events):\n")
for _ in range(2000):
    event = gen.generate_event()
    state.update(event)

print(f"Total Events: {state.total_events:,}")
print(f"Overall Pass Rate: {state.total_passes / state.total_events * 100:.2f}%\n")

print("Per-Wafer Yield:")
for wafer_id in sorted(state.wafer_pass_counts.keys())[:5]:  # Show first 5
    yield_pct = state.get_wafer_yield(wafer_id) * 100
    total = state.wafer_pass_counts.get(wafer_id, 0) + state.wafer_fail_counts.get(wafer_id, 0)
    print(f"  {wafer_id}: {yield_pct:.1f}% ({total} tests)")

### 📝 What's Happening in This Code?

**Purpose:** Maintain running state for real-time aggregations

**Key Points:**
- **Stateful Operations**: Keep per-wafer counters (not recomputing from scratch)
- **Incremental Updates**: O(1) per event vs O(N) batch scan
- **Memory Management**: Production uses bounded state with TTL (time-to-live)

**Why This Matters:** Intel tracks 50K wafers concurrently (10GB RAM). State checkpointed to RocksDB every 1 minute.

In [ ]:
# Simulate streaming data collection for visualization
gen = TestEventGenerator(events_per_second=1000, failure_rate=0.08)
processor = StatefulStreamProcessor()

time_series = []
for _ in range(60000):  # 60 seconds
    event = gen.generate_event()
    processor.process_event(event)
    
    # Sample every 100 events for plotting
    if processor.state.total_events % 100 == 0:
        metrics = {
            'timestamp': event.event_time,
            'total_events': processor.state.total_events,
            'pass_rate': processor.state.total_passes / processor.state.total_events,
            'unique_wafers': len(processor.state.wafer_pass_counts),
            'recent_latency': event.latency_ms
        }
        time_series.append(metrics)

df_metrics = pd.DataFrame(time_series)
df_metrics['seconds'] = (df_metrics['timestamp'] - df_metrics['timestamp'].min()).dt.total_seconds()

# Create dashboard
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Real-Time Streaming Dashboard', fontsize=16, fontweight='bold')

# Throughput
axes[0, 0].plot(df_metrics['seconds'], df_metrics['total_events'], linewidth=2, color='#2ecc71')
axes[0, 0].set_xlabel('Time (seconds)')
axes[0, 0].set_ylabel('Total Events')
axes[0, 0].set_title('Event Throughput')
axes[0, 0].grid(True, alpha=0.3)

# Pass rate
axes[0, 1].plot(df_metrics['seconds'], df_metrics['pass_rate'] * 100, linewidth=2, color='#3498db')
axes[0, 1].axhline(y=90, color='red', linestyle='--', label='Target 90%')
axes[0, 1].set_xlabel('Time (seconds)')
axes[0, 1].set_ylabel('Pass Rate (%)')
axes[0, 1].set_title('Real-Time Yield')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Latency
axes[1, 0].plot(df_metrics['seconds'], df_metrics['recent_latency'], linewidth=2, color='#e74c3c')
axes[1, 0].set_xlabel('Time (seconds)')
axes[1, 0].set_ylabel('Latency (ms)')
axes[1, 0].set_title('Processing Latency')
axes[1, 0].grid(True, alpha=0.3)

# Wafer discovery
axes[1, 1].plot(df_metrics['seconds'], df_metrics['unique_wafers'], linewidth=2, color='#9b59b6')
axes[1, 1].set_xlabel('Time (seconds)')
axes[1, 1].set_ylabel('Unique Wafers')
axes[1, 1].set_title('Wafer Discovery Rate')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Processed {df_metrics['total_events'].iloc[-1]:,} events in 60 seconds")
print(f"   Throughput: {df_metrics['total_events'].iloc[-1]/60:.0f} events/sec")
print(f"   Final Pass Rate: {df_metrics['pass_rate'].iloc[-1]*100:.2f}%")

### 📝 What's Happening in This Code?

**Purpose:** Create real-time dashboard visualization of streaming metrics

**Key Points:**
- **Time-Series Metrics**: Sample every 100 events (production: every 1-5 seconds)
- **4-Panel Dashboard**: Throughput, yield, latency, discovery (standard streaming metrics)
- **WebSocket Updates**: Production uses WebSockets to push to browser (not shown here)
- **Alert Visualization**: Red threshold line shows target yield

**Why This Matters:** Intel's dashboard updates every 1 second with 1000+ concurrent users (Grafana + InfluxDB). Yield drops visible immediately → fab response in <1 minute.

## 6. Real-Time Dashboard Visualization

### 📝 What's Happening in This Code?

**Purpose:** Add real-time anomaly detection with automated alerting

**Key Points:**
- **Periodic Checks**: Check every 100 events (production: every 1-5 seconds)
- **Threshold-Based**: Alert if wafer yield <85% with ≥10 tests (statistical significance)
- **Alert Callback**: Trigger external actions (PagerDuty, Slack, email)
- **Alert History**: Track all alerts for post-mortem analysis

**Why This Matters:** Intel detects yield drops in 30 seconds (vs 4 hours batch). Alerts trigger fab investigation saving $50M/year in scrap.

In [ ]:
class StatefulStreamProcessor:
    """Stateful processor with anomaly detection and alerting"""
    
    def __init__(self, alert_callback: Optional[Any] = None):
        self.state = StreamState()
        self.alert_callback = alert_callback
        self.alert_history: List[Dict[str, Any]] = []
    
    def process_event(self, event: TestEvent):
        self.state.update(event)
        
        # Check for anomalies every 100 events
        if self.state.total_events % 100 == 0:
            anomalies = self._detect_anomalies(threshold=0.85)
            if anomalies and self.alert_callback:
                alert = {
                    'timestamp': event.processing_time,
                    'type': 'LOW_YIELD',
                    'wafers': anomalies,
                    'message': f"{len(anomalies)} wafer(s) below 85% yield"
                }
                self.alert_history.append(alert)
                self.alert_callback(alert)
    
    def _detect_anomalies(self, threshold: float) -> List[str]:
        """Detect wafers with abnormally low yield"""
        anomalies = []
        for wafer_id in self.state.wafer_pass_counts.keys():
            total = (self.state.wafer_pass_counts.get(wafer_id, 0) + 
                     self.state.wafer_fail_counts.get(wafer_id, 0))
            if total >= 10:
                yield_rate = self.state.get_wafer_yield(wafer_id)
                if yield_rate < threshold:
                    anomalies.append(wafer_id)
        return anomalies

# Test with alerts
def alert_handler(alert):
    print(f"\n🚨 ALERT: {alert['message']}")
    print(f"   Wafers: {', '.join(alert['wafers'][:3])}...")
    print(f"   Time: {alert['timestamp'].strftime('%H:%M:%S')}")

gen = TestEventGenerator(events_per_second=500, failure_rate=0.12)
processor = StatefulStreamProcessor(alert_callback=alert_handler)

print("🔄 Processing 5000 events with anomaly detection...\n")
for _ in range(5000):
    event = gen.generate_event()
    processor.process_event(event)

print(f"\n✅ Processed {processor.state.total_events:,} events")
print(f"   Overall Pass Rate: {processor.state.total_passes / processor.state.total_events * 100:.2f}%")
print(f"   Alerts Triggered: {len(processor.alert_history)}")

## 5b. Anomaly Detection with Alerts

## 6. Real-World Projects

## 7. Real-World Projects 🚀

### Post-Silicon Validation Projects

#### **Project 1: Intel Real-Time Yield Monitor ($50M/year)**

**Objective:** Build streaming pipeline to detect yield drops within 30 seconds across 10,000 testers

**Success Metrics:**
- Process 5M events/second with <100ms p99 latency
- Detect 2% yield drop within 30 seconds (current: 4 hours batch)
- 99.99% uptime (4 nines SLA)

**Business Value:** $50M/year prevented scrap (early detection stops bad lots)

**Tech Stack:**
- **Ingestion**: Kafka (100 partitions, 7-day retention, 3× replication)
- **Processing**: Apache Flink (50-node cluster, event time processing)
- **Storage**: Cassandra (time-series metrics, 1-year retention)
- **Alerting**: PagerDuty integration (SMS/email to fab engineers)
- **Visualization**: Grafana dashboards (1000+ concurrent users)

**Implementation Details:**
- **Partitioning**: By `tester_id` (10K testers → 100 partitions = 100 testers/partition)
- **Windows**: 1-minute tumbling windows (per tester, per wafer, per lot)
- **Aggregations**: Count, pass rate, mean/stddev of parametric values
- **Anomaly Detection**: 3-sigma alerts (yield drop >3 std deviations from baseline)
- **Spatial Correlation**: Detect wafer map patterns (same die_x, die_y failures across wafers)
- **State**: 50K wafers concurrently tracked (10GB RAM), checkpointed every 1 minute to RocksDB
- **Exactly-Once**: Kafka transactions + Flink checkpoints (no duplicate alerts)

**Features:**
- Per-tester yield dashboards (real-time, 1-second updates)
- Per-wafer spatial maps (heatmaps updated every 10 seconds)
- Automated email alerts (fab managers within 30 seconds of detection)
- Historical playback (debug past yield drops)
- Root cause analysis (correlate with equipment telemetry)

---

#### **Project 2: NVIDIA Adaptive Test Binning ($45M/year)**

**Objective:** Real-time ML model scoring to optimize GPU binning (50K devices/minute)

**Success Metrics:**
- <100ms model inference latency per device
- 5% yield improvement via adaptive binning
- Support 500 concurrent test streams

**Business Value:** $45M/year (1% yield = $9M for $900M revenue/site × 5 sites)

**Tech Stack:**
- **Ingestion**: Kafka (parametric test results, 50K msgs/min)
- **Processing**: Spark Structured Streaming (micro-batches, 1-second triggers)
- **ML Serving**: TensorFlow Serving (GPU inference, 10ms p99 latency)
- **Feedback Loop**: Update test systems with new bin assignments
- **Monitoring**: Prometheus + Grafana (lag, throughput, model latency)

**Implementation Details:**
- **Feature Engineering**: 500+ parametric tests → 50 features (PCA dimensionality reduction)
- **Model**: XGBoost classifier (5 bin classes: GeForce RTX 4090 → RTX 4060)
- **Serving**: TensorFlow Serving on GPU (batch inference 100 devices at a time)
- **A/B Testing**: Compare old vs new binning (30-day trials, track yield/revenue)
- **Retraining**: Daily model updates (ingest previous day's data, retrain overnight)
- **Cold Start**: Fallback to rule-based binning if model unavailable

**Features:**
- Real-time bin prediction (parametric test results → bin in <100ms)
- Confidence scores (reject low-confidence predictions, send to manual review)
- Bin boundary tuning (optimize yield vs performance targets)
- Multi-site deployment (8 fabs worldwide, centralized model serving)
- ROI tracking (track revenue per bin, optimize for maximum revenue)

---

#### **Project 3: Qualcomm Multi-Site Correlation Engine ($30M/year)**

**Objective:** Detect correlated failures across 8 global fabs in real-time

**Success Metrics:**
- Ingest from 8 sites (3 continents, 24/7 operations)
- Detect spatial/temporal correlation within 5 minutes
- Reduce MTTR from 12 hours to 2 hours

**Business Value:** $30M/year (faster root cause = less scrap, faster fixes)

**Tech Stack:**
- **Ingestion**: Multi-region Kafka (cross-datacenter replication with MirrorMaker 2.0)
- **Processing**: Apache Flink CEP (Complex Event Processing, pattern matching)
- **Correlation**: Sliding windows (10-minute window, 5-minute slide)
- **Storage**: Elasticsearch (searchable event logs, 90-day retention)
- **Alerting**: Automated Jira ticket creation (root cause library lookup)

**Implementation Details:**
- **CEP Patterns**: Define patterns like "3 wafers with >10% yield drop in same lot within 1 hour"
- **Geospatial Clustering**: Same fab floor, same tester, same process tool
- **Temporal Clustering**: Failures within 1-hour window (across sites)
- **Root Cause Library**: 500+ known failure patterns (match incoming events)
- **Cross-Site Latency**: 50-200ms (US ↔ Asia ↔ EU)
- **Exactly-Once**: Critical for avoiding duplicate Jira tickets

**Features:**
- Multi-site correlation (detect if same issue happening at multiple fabs)
- Automated root cause lookup (match pattern → suggest likely cause)
- Jira integration (create ticket, assign to correct team, include diagnostics)
- Historical search (Elasticsearch query interface for post-mortem analysis)
- Network resilience (each region can operate independently if others down)

---

#### **Project 4: AMD Equipment Health Monitoring ($25M/year)**

**Objective:** Predict tester failures from real-time telemetry (temp, vibration, power)

**Success Metrics:**
- Ingest 100Hz sensor data from 5,000 testers
- Predict failure 2 hours before (80% accuracy)
- Reduce unplanned downtime by 70%

**Business Value:** $25M/year (downtime costs $50K/hour per tester × 500 hours saved/year × 1000 testers)

**Tech Stack:**
- **Ingestion**: MQTT → Kafka bridge (IoT protocol, lightweight)
- **Processing**: Apache Flink (stateful processing, 5-minute windows)
- **ML**: Isolation Forest (anomaly detection, scikit-learn)
- **Alerting**: ServiceNow integration (predictive maintenance tickets)
- **Storage**: InfluxDB (time-series sensor data, 1-year retention)

**Implementation Details:**
- **Sensors**: Temperature (10 zones), vibration (3-axis accelerometer), power (voltage/current)
- **Sampling**: 100Hz (100 samples/second per sensor per tester)
- **Windowing**: 5-minute sliding windows (1-minute slide)
- **Feature Engineering**: Mean, stddev, max, min, rate of change per window
- **Anomaly Detection**: Isolation Forest on 30+ features (multivariate outliers)
- **Alert Threshold**: 0.8 anomaly score (tuned to 80% accuracy, 10% false positive rate)
- **Predictive Horizon**: 2 hours (median time from anomaly → failure)

**Features:**
- Real-time tester health dashboards (traffic light: green/yellow/red)
- Predictive maintenance scheduling (integrate with MES calendar)
- Historical playback (analyze past failures, improve model)
- Multi-sensor correlation (temperature spike + vibration = bearing failure)
- Cost avoidance tracking (track prevented downtime events)

---

### General AI/ML Projects

#### **Project 5: Uber Real-Time Surge Pricing ($100M/year)**
- **Objective**: Dynamic pricing based on rider demand + driver supply
- **Tech**: Kafka + Flink, 1M events/second, <100ms latency
- **Features**: Geohash clustering, demand prediction, price optimization
- **Value**: $100M/year increased revenue (optimal pricing)

#### **Project 6: Netflix Viewing Quality Monitor ($80M/year)**
- **Objective**: Stream video quality metrics → real-time CDN routing decisions
- **Tech**: Kafka + Spark Streaming, 100K concurrent streams
- **Features**: Buffering detection, bitrate optimization, CDN failover
- **Value**: $80M/year reduced CDN costs + improved customer satisfaction

#### **Project 7: Airbnb Fraud Detection ($60M/year)**
- **Objective**: Stream booking events → ML fraud scoring → block in <1 second
- **Tech**: Kafka + Flink CEP, 10K bookings/minute
- **Features**: Rule engine + ML model, graph fraud detection, risk scoring
- **Value**: $60M/year prevented fraud losses

#### **Project 8: PayPal Transaction Risk Scoring ($150M/year)**
- **Objective**: Stream payment events → risk model → approve/reject in 200ms
- **Tech**: Kafka + Flink, 50K transactions/second, 99.999% uptime
- **Features**: Real-time feature engineering, ensemble models, adaptive thresholds
- **Value**: $150M/year (fraud prevention + reduced false declines)

---

**Total Business Impact: $595M/year** across all projects

## 7. Key Takeaways 🎓

### When to Use Stream Processing

✅ **Use Streaming When:**
- Latency matters (seconds/minutes, not hours)
- Continuous data arrival
- Real-time actions (alerts, feedback loops)
- Event-driven business logic

❌ **Use Batch When:**
- Latency acceptable (hours/days)
- Complete historical data needed
- Complex multi-pass algorithms

### Technical Patterns

**Windowing:**
- Tumbling: Non-overlapping (dashboards)
- Sliding: Overlapping (moving averages)
- Session: Gap-based (user sessions)

**State Management:**
- Bounded state (limit memory with TTL)
- Checkpointing (RocksDB for fault tolerance)
- Exactly-once semantics (Kafka + Flink)

**Production Best Practices:**
- Kafka: 3+ brokers, replication factor 3
- Monitoring: Prometheus + Grafana (lag, throughput, latency)
- Schema Registry: Avro/Protobuf (backward compatibility)
- Testing: Flink MiniCluster, Kafka TestContainers

---

**You now understand real-time stream processing!** 🎉

**Next:** 096: Batch Processing at Scale